In [1]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions import *

In [ ]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [ ]:
nodes, edges, s, n, m = create_problem_instance(path)
is_splex(nodes = nodes, plex_number = 2, s=s)

# Construction heuristic

In [ ]:
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes,edges,s)
print("number of plexes: "+ str(len(constr_nodes["splex"].unique())))
print("score: " +str(constr_score))
print(constr_edges)

# Randomized Construction heuristic

In [ ]:
rand_nodes, rand_edges, rand_score = randomized_greedy(nodes, edges, s, alpha = 0.75)
print("number of plexes: "+ str(len(rand_nodes["splex"].unique())))
print("score: " +str(rand_score))

# extract 1 node neighborhood

In [ ]:
extract_nodes, extract_edges, extract_score = extract1node(constr_nodes, constr_edges, constr_score, s, step = "best")
print("number of plexes: "+ str(len(extract_nodes["splex"].unique())))
print("score: " +str(extract_score))

# merge 2 plexes neighborhood

In [ ]:
merged_nodes, merged_edges, merged_score = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = True)
print("number of plexes: "+ str(len(merged_nodes["splex"].unique())))
print("score: " +str(merged_score))


In [ ]:
merged_nodes2, merged_edges2, merged_score2 = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = False)
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

## do it again on the new best solution

In [ ]:
merged_nodes3, merged_edges3, merged_score3 = merge2plexes(merged_nodes, merged_edges, merged_score, s, step = "best")
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

# einmal alles (mit scharf)

In [ ]:
instance_type = "inst_competition/"#"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur051_n_300_m_20122" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"
nodes, edges, s, n, m = create_problem_instance(path)
print(edges)
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes, edges, s)
write_solution(constr_edges, problem_instance, algorithm = "construction")
print(constr_edges)

In [2]:
def vnd(solution: Solution, neighborhoods: list[Neighborhood], max_minutes = 60) -> Solution:
    current_best: Solution = solution
    
    start = time.time()

    i = 0 # First neighborhood
    while i < len(neighborhoods):
        current_neighborhood: Neighborhood = neighborhoods[i]
        local_optimum = current_neighborhood.get_improvement(current_best, "first")

        print("i: " + str(i) + ", " + str(local_optimum.get_weight()))
        if not valid(local_optimum):
            break
            
        if not consistent(local_optimum):
            break

        if local_optimum.get_weight() < current_best.get_weight():
            current_best = local_optimum
            i = 0
        else:
            i += 1
        
        if (time.time()-start) > (max_minutes * 60):
            break

    return current_best

In [3]:
plex_merge_neighborhood = Neighborhood(lambda sol, step: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step, ))
add_edge_neighborhood = Neighborhood(lambda sol, step: add_edge_within_plex(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
remove_edge_neighborhood = Neighborhood(lambda sol, step: remove_edge(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step, skip_invalid=True))
extract_node_neighborhood = Neighborhood(lambda sol, step: extract1node(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))

In [ ]:
problem = Problem(path)
solution = grasp(problem, plex_merge_neighborhood, 20)
print(solution.get_weight())

In [ ]:
#problem = Problem("data/inst_competition/heur051_n_300_m_20122.txt")
#initial_solution = problem.get_heuristic_solution()
#print(valid(initial_solution))
#print(initial_solution.get_weight())
#print(initial_solution.get_nodes())
#write_solution(initial_solution.get_edges(), "heur051_n_300_m_20122", "construction")
#local_optimum = vnd(initial_solution, [add_edge_neighborhood, remove_edge_neighborhood, extract_node_neighborhood, plex_merge_neighborhood])
print(valid(local_optimum))
write_solution(initial_solution.get_edges(), "heur051_n_300_m_20122", "vnd")

In [ ]:
#plex_merge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#add_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#remove_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#extract_node_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)

local_optimum.get_edges()

In [4]:
instances = ["heur049_n_300_m_17695", "heur050_n_300_m_19207", "heur051_n_300_m_20122"]
problems = dict()
heuristics = dict()
randomized = dict()
vnds = dict()
for instance in instances:
    curr_problem = Problem("data/inst_competition/" + instance + ".txt")
    problems[instance] = curr_problem
    
    curr_heuristic = curr_problem.get_heuristic_solution()
    heuristics[instance] = curr_heuristic
    write_solution(curr_heuristic.get_edges(), instance, "construction")
    
    #curr_randomized = None
    #for i in range(2):
    #    curr = curr_problem.get_randomized_solution()
    #    if (randomized == None) or (curr.get_weight() < curr.get_weight()):
    #        randomized = curr
    #randomized[instance] = curr_randomized
    #write_solution(curr_randomized.get_edges(), instance, "randomized")

    curr_vnd = vnd(curr_heuristic, [add_edge_neighborhood, remove_edge_neighborhood, extract_node_neighborhood, plex_merge_neighborhood], 2)
    vnds[instance] = curr_vnd
    write_solution(curr_vnd.get_edges(), instance, "vnd")

<class 'pandas.core.frame.DataFrame'>
      n1  n2  e  w
4953  18  25  0 -2
<class 'pandas.core.frame.DataFrame'>
        n1   n2  e  w
0        1    2  0 -7
1        1    3  0 -1
2        1    4  0 -1
3        1    5  0 -5
4        1    6  0 -7
...    ...  ... .. ..
44845  297  299  0 -3
44846  297  300  0 -4
44847  298  299  0 -1
44848  298  300  0 -2
44849  299  300  0 -4

[44850 rows x 4 columns]


ValueError: Can only compare identically-labeled Series objects